In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import sys
sys.path.append("../")

# Get Metadata

In [2]:
df = pd.read_csv('../data/exams.csv')
df.head()

,exam_id,age,is_male,nn_predicted_age,1dAVb,RBBB,LBBB,SB,ST,AF,patient_id,death,timey,normal_ecg,trace_file
0,1169160,38,True,40.160484,False,False,False,False,False,False,523632,False,2.098628,True,exams_part13.hdf5
1,2873686,73,True,67.059440,False,False,False,False,False,False,1724173,False,6.657529,False,exams_part13.hdf5
2,168405,67,True,79.621740,False,False,False,False,False,True,51421,False,4.282188,False,exams_part13.hdf5
3,271011,41,True,69.750260,False,False,False,False,False,False,1737282,False,4.038353,True,exams_part13.hdf5
4,384368,73,True,78.873460,False,False,False,False,False,False,331652,False,3.786298,False,exams_part13.hdf5


# Create the peaks summary file if doesn't exist

Run 
`summary_frame, mask_df, data_array, exam_ids = create_summary_frame(data_array, exam_ids)` from `one_beat.py`

OR

`python one_beat.py` will create these files:
```
"../data/beats_summary_frame.csv"
"../data/selected_traces_part16.npy"

```

In [3]:
df_summ = pd.read_csv("../data/beats_summary_frame.csv")
if 'Unnamed: 0' in df_summ.columns:
    df_summ.drop(columns=['Unnamed: 0'], inplace=True)
# df_summ.drop(columns=['retain_subject'], inplace=True)    
df_summ.head()

,subject,channel,n_peaks,inter_beat_sd,peaks,mode_n_peaks,mode_count,min_peak,max_peak,hr,avg_inter_beat_sd,retain_subject,new_subject_id
0,0,0,9.0,9.096703,[ 211 670 1118 1560 2002 2443 2885 3314 3743],9.0,10,211.0,3743.0,55.663204,9.104809,True,0.0
1,0,1,9.0,9.137833,[ 212 671 1119 1561 2004 2444 2886 3316 3744],9.0,10,212.0,3744.0,55.663204,9.104809,True,0.0
2,0,2,3.0,656.500000,[1123 1565 3320],9.0,10,1123.0,3320.0,55.663204,9.104809,True,0.0
3,0,3,9.0,9.124144,[ 211 670 1118 1560 2003 2444 2885 3315 3743],9.0,10,211.0,3743.0,55.663204,9.104809,True,0.0
4,0,4,9.0,9.096703,[ 210 669 1117 1559 2001 2443 2884 3313 3742],9.0,10,210.0,3742.0,55.663204,9.104809,True,0.0


In [4]:
df_summ.loc[:, "peaks"] = df_summ[
    "peaks"
].apply(lambda x: [int(item) for item in x.replace('[', '').replace(']', '').split()])

In [5]:
df_summ.head()

,subject,channel,n_peaks,inter_beat_sd,peaks,mode_n_peaks,mode_count,min_peak,max_peak,hr,avg_inter_beat_sd,retain_subject,new_subject_id
0,0,0,9.0,9.096703,"[211, 670, 1118, 1560, 2002, 2443, 2885, 3314,...",9.0,10,211.0,3743.0,55.663204,9.104809,True,0.0
1,0,1,9.0,9.137833,"[212, 671, 1119, 1561, 2004, 2444, 2886, 3316,...",9.0,10,212.0,3744.0,55.663204,9.104809,True,0.0
2,0,2,3.0,656.500000,"[1123, 1565, 3320]",9.0,10,1123.0,3320.0,55.663204,9.104809,True,0.0
3,0,3,9.0,9.124144,"[211, 670, 1118, 1560, 2003, 2444, 2885, 3315,...",9.0,10,211.0,3743.0,55.663204,9.104809,True,0.0
4,0,4,9.0,9.096703,"[210, 669, 1117, 1559, 2001, 2443, 2884, 3313,...",9.0,10,210.0,3742.0,55.663204,9.104809,True,0.0


In [6]:
data_array = np.load("../data/selected_traces_part16.npy")

In [29]:
df_meta = pd.read_csv("../data/average_beat_metadata.csv")
df_meta.shape

(6044, 5)

In [30]:
df_meta.head()

,exam_id,data_arr_idx,start_beat,end_beat,channel_used
0,1750214,0,1893,2358,0
1,1688724,1,1928,2288,1
2,1771393,2,1939,2266,0
3,1793030,3,1934,2275,0
4,1717689,4,1973,2197,1


In [32]:
df_meta['data_arr_idx'].max()

np.int64(19996)

In [31]:
channel_dict = {}
for idx, channel in zip(df_meta['data_arr_idx'].values, df_meta['channel_used'].values):
    channel_dict[int(idx)] = int(channel)
max(channel_dict.keys())

19996

In [10]:
# This should give no error
assert(
    df_summ[df_summ['retain_subject']]['subject'].nunique() == len(df_meta)
)

In [11]:
df_summ['subject'].max(), df_summ['new_subject_id'].max(), df_meta['data_arr_idx'].max()

(np.int64(20000), np.float64(6043.0), np.int64(19996))

In [12]:
df_meta = df_summ[['subject', 'channel', 'n_peaks', 'peaks', 'new_subject_id']].merge(
    df_meta.reset_index()[['index', 'exam_id', 'channel_used']],
    left_on='new_subject_id',
    right_on='index',
).drop(columns='index')

In [13]:
df_meta['subject'].nunique()

6044

In [14]:
df_meta.head()

,subject,channel,n_peaks,peaks,new_subject_id,exam_id,channel_used
0,0,0,9.0,"[211, 670, 1118, 1560, 2002, 2443, 2885, 3314,...",0.0,1750214,0
1,0,1,9.0,"[212, 671, 1119, 1561, 2004, 2444, 2886, 3316,...",0.0,1750214,0
2,0,2,3.0,"[1123, 1565, 3320]",0.0,1750214,0
3,0,3,9.0,"[211, 670, 1118, 1560, 2003, 2444, 2885, 3315,...",0.0,1750214,0
4,0,4,9.0,"[210, 669, 1117, 1559, 2001, 2443, 2884, 3313,...",0.0,1750214,0


In [28]:
max(channel_dict.keys())

19996

In [33]:
peaks_dict = {}

for idx, channel in channel_dict.items():
    try:
        peaks_dict[idx] = df_meta[
        (df_meta['subject'] == idx) &
        (df_meta['channel'] == channel)
    ]['peaks'].values[0]
    except:
        print (idx, channel)
        
peaks_dict

0 0
1 1
2 0
3 0
4 1
12 0
16 0
20 0
23 0
26 0
29 0
31 0
41 0
42 0
43 0
44 1
45 1
49 0
52 0
55 0
58 0
69 0
71 0
74 0
80 0
82 1
83 0
89 0
92 1
94 0
97 0
99 0
108 0
110 0
112 0
115 0
125 0
133 0
134 1
139 0
140 0
141 0
153 0
154 0
155 0
156 0
158 0
159 0
161 0
162 0
163 0
167 0
169 0
170 0
173 0
176 0
177 0
179 0
190 1
191 0
196 0
198 0
212 0
213 0
217 0
220 0
221 0
223 0
229 0
232 1
233 0
241 0
251 0
252 0
255 0
257 0
263 0
265 0
268 0
272 0
274 0
276 0
277 1
280 0
281 0
294 0
298 0
300 0
303 0
304 0
311 1
312 0
313 0
316 0
319 0
320 0
321 0
322 0
323 0
325 0
330 1
332 0
342 0
346 0
347 0
350 0
355 0
365 0
370 0
376 0
377 0
378 0
383 0
393 0
396 0
399 0
402 0
416 1
421 0
422 0
423 0
425 1
427 0
428 0
431 0
433 0
436 1
439 0
448 0
452 0
454 1
458 0
468 0
475 0
478 0
479 1
480 0
483 0
484 0
490 0
491 0
498 0
499 1
500 0
506 0
510 0
514 0
515 0
517 0
521 0
522 0
523 0
527 0
538 0
542 0
543 0
544 0
546 0
548 0
551 0
555 1
556 0
557 1
562 0
564 0
574 0
583 0
588 1
591 0
592 0
597 0
598 0
603 0

{}

In [34]:
df_meta[
    (df_meta['subject'] == 0) &
    (df_meta['channel'] == 0)
]

KeyError: 'subject'

In [19]:
df_meta.head(20)

,subject,channel,n_peaks,peaks,new_subject_id,exam_id,channel_used
0,0,0,9.0,"[211, 670, 1118, 1560, 2002, 2443, 2885, 3314,...",0.0,1750214,0
1,0,1,9.0,"[212, 671, 1119, 1561, 2004, 2444, 2886, 3316,...",0.0,1750214,0
2,0,2,3.0,"[1123, 1565, 3320]",0.0,1750214,0
3,0,3,9.0,"[211, 670, 1118, 1560, 2003, 2444, 2885, 3315,...",0.0,1750214,0
4,0,4,9.0,"[210, 669, 1117, 1559, 2001, 2443, 2884, 3313,...",0.0,1750214,0
5,0,5,9.0,"[214, 673, 1121, 1563, 2005, 2446, 2888, 3318,...",0.0,1750214,0
6,0,6,9.0,"[213, 672, 1120, 1562, 2004, 2445, 2887, 3317,...",0.0,1750214,0
7,0,7,9.0,"[217, 676, 1124, 1566, 2008, 2449, 2891, 3320,...",0.0,1750214,0
8,0,8,9.0,"[216, 675, 1124, 1566, 2008, 2449, 2890, 3320,...",0.0,1750214,0
9,0,9,9.0,"[208, 667, 1115, 1557, 2000, 2440, 2882, 3312,...",0.0,1750214,0


In [19]:
i = 0
trace = data_array[:1, :, :]
trace.shape

(1, 4096, 12)

In [24]:
df_meta.groupby('subject').channel_used.max().reset_index()

,subject,channel_used
0,0,0
1,1,1
2,2,0
3,3,0
4,4,1
...,...,...
6039,19979,0
6040,19983,0
6041,19986,0
6042,19991,0
